In [2]:
import folium
from folium.plugins import HeatMap
from haversine import haversine, Unit, haversine_vector
import pandas as pd
from IPython.display import display

In [3]:
osm_data = pd.read_json('data/preprocessed-osm-data.json.gz')
osm_data

,lat,lon,timestamp,amenity,name,tags,cuisine,qid
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ...",coffee_shop,Q37158
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...,None,None
2,49.373423,-123.291894,2016-10-10T02:14:29.000-07:00,toilets,None,{},None,None
3,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,None,{},None,None
4,49.370898,-123.280448,2015-05-03T00:42:25.000-07:00,place_of_worship,St. Monica's Anglican Church,"{'addr:housenumber': '6404', 'addr:street': 'W...",None,None
...,...,...,...,...,...,...,...,...
17713,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{},None,None
17714,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'},japanese,None
17715,49.276443,-122.790138,2013-03-26T23:45:50.000-07:00,parking,None,{},None,None
17716,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne...",None,None


In [4]:
def nonRestaurestaurant(): 
    return [
    'ATLAS_clean_room' ,'EVSE','Observation Platform' ,'Pharmacy','animal_shelter','arts_centre','atm','atm;bank',
    'bank','bench','bicycle_parking','bicycle_rental','bicycle_repair_station','car_rental','car_rep','car_sharing',
    'car_wash','casino','charging_station','childcare','chiropractor','cinema','clinic','clock','college',
    'community_centre','compressed_air','conference_centre','construction','courthouse','cram_school','dentist',
    'events_venue','family_centre','ferry_terminal','fire_station','first_aid','fountain','fuel','gambling','gym',
    'healthcare','hospital','housing co-op','hunting_stand','kindergarten','language_school','leisure','letter_box',
    'library','loading_dock','lobby','lounge','luggage_locker','marketplace','meditation_centre','monastery',
    'money_transfer','motorcycle_parking','motorcycle_rental','music_school','nightclub','nursery',
    'office|financial','park','parking','parking_entrance','parking_space','payment_terminal','pharmacy',
    'photo_booth','place_of_worship','playground','police','post_box','post_depot','post_office','prep_school',
    'public_bookcase','public_building','ranger_station','recycling','research_institute','safety',
    'sanitary_dump_station','school','science','scrapyard','seaplane terminal','shelter','shop|clothes','shower',
    'smoking_area','social_centre','social_facility','spa','storage','storage_rental','stripclub','studio',
    'taxi','telephone','theatre','toilets','townhall','training','trash','trolley_bay','university','vacuum_cleaner',
    'vending_machine','driving_school','veterinary','waste_basket','waste_disposal','waste_transfer_station',
    'water_point','watering_place','workshop''bureau_de_change','bus_station','internet_cafe', 'doctors',
    'dojo','bureau_de_change', 'boat_rental', 'workshop', 'drinking_water'

    ]



In [5]:
# get all amenities
amenity=osm_data['amenity']

# filter restuarants from non restaurants
restaurant=list(dict.fromkeys([i for i in amenity if i not in nonRestaurestaurant()]))
restaurant

['cafe',
 'fast_food',
 'bbq',
 'restaurant',
 'pub',
 'bar',
 'food_court',
 'ice_cream',
 'bistro',
 'juice_bar',
 'disused:restaurant',
 'biergarten']

In [6]:
# filter restuarants from non restaurants in osm_data
osm_data=osm_data[osm_data['amenity'].isin(restaurant)]
osm_data

,lat,lon,timestamp,amenity,name,tags,cuisine,qid
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ...",coffee_shop,Q37158
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...,None,None
3,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,None,{},None,None
13,49.126650,-123.182470,2020-03-30T09:08:51.000-07:00,restaurant,Best Bite Indian Cuisine,"{'addr:housenumber': '10-3891', 'phone': '+1-6...",indian,None
16,49.283192,-123.109050,2015-12-18T21:41:07.000-08:00,pub,The Cambie,"{'toilets:wheelchair': 'no', 'wheelchair': 'li...",None,None
...,...,...,...,...,...,...,...,...
17712,49.250408,-123.076261,2017-07-08T05:22:57.000-07:00,restaurant,House of Dosas,"{'addr:housenumber': '1391', 'phone': '+1-604-...",indian,None
17713,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{},None,None
17714,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'},japanese,None
17716,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne...",None,None


In [7]:
# chain restaurant data using qid
chain_qids=pd.read_json('data/chain-restaurant-qids.json')
chain_qids

,qid,is_chain_restaurant
0,Q175106,1
1,Q3472954,1
2,Q894578,1
3,Q38076,1
4,Q244457,1
5,Q524757,1
6,Q177054,1
7,Q2996960,1
8,Q2818848,1
9,Q191615,1


In [8]:
# merge osm_data and chain_qid to identify chain restaurants in 
osm_data=osm_data.merge(chain_qids, how='left', on='qid')
osm_data['is_chain_restaurant']=osm_data.is_chain_restaurant.fillna(0)
osm_data

,lat,lon,timestamp,amenity,name,tags,cuisine,qid,is_chain_restaurant
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ...",coffee_shop,Q37158,0.0
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...,None,None,0.0
2,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,None,{},None,None,0.0
3,49.126650,-123.182470,2020-03-30T09:08:51.000-07:00,restaurant,Best Bite Indian Cuisine,"{'addr:housenumber': '10-3891', 'phone': '+1-6...",indian,None,0.0
4,49.283192,-123.109050,2015-12-18T21:41:07.000-08:00,pub,The Cambie,"{'toilets:wheelchair': 'no', 'wheelchair': 'li...",None,None,0.0
...,...,...,...,...,...,...,...,...,...
5145,49.250408,-123.076261,2017-07-08T05:22:57.000-07:00,restaurant,House of Dosas,"{'addr:housenumber': '1391', 'phone': '+1-604-...",indian,None,0.0
5146,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{},None,None,0.0
5147,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'},japanese,None,0.0
5148,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne...",None,None,0.0


In [9]:
def make_matrix(location, size):
    return [location for i in range(size)]

In [10]:
osm_data[osm_data.is_chain_restaurant==1].shape

(745, 9)

In [11]:
# default initializations
location1 = [49.2768,  -122.9180] #SFU burnaby
location2 = [49.284478, -123.112349]  #SFU Vancouver
dist=5

In [12]:
size=osm_data.lat.size
    
#convert locations list to matrix for haversine_vector(), distance calculation
location1_matrix=make_matrix(location1, size)
location2_matrix=make_matrix(location2, size)

#distance between location 1 and osm_data locations
osm_data['dist1']=haversine_vector(
osm_data[['lat', 'lon']].values.tolist(), [location1 for i in range(size)], 
Unit.KILOMETERS
)

#distance between location 2 and osm_data locations
osm_data['dist2']=haversine_vector(
    osm_data[['lat', 'lon']].values.tolist(), [location2 for i in range(size)], 
    Unit.KILOMETERS
)
osm_data

,lat,lon,timestamp,amenity,name,tags,cuisine,qid,is_chain_restaurant,dist1,dist2
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ...",coffee_shop,Q37158,0.0,15.176974,2.805051
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...,None,None,0.0,15.172818,2.789539
2,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,None,{},None,None,0.0,4.259253,11.725696
3,49.126650,-123.182470,2020-03-30T09:08:51.000-07:00,restaurant,Best Bite Indian Cuisine,"{'addr:housenumber': '10-3891', 'phone': '+1-6...",indian,None,0.0,25.455232,18.274133
4,49.283192,-123.109050,2015-12-18T21:41:07.000-08:00,pub,The Cambie,"{'toilets:wheelchair': 'no', 'wheelchair': 'li...",None,None,0.0,13.876889,0.278744
...,...,...,...,...,...,...,...,...,...,...,...
5145,49.250408,-123.076261,2017-07-08T05:22:57.000-07:00,restaurant,House of Dosas,"{'addr:housenumber': '1391', 'phone': '+1-604-...",indian,None,0.0,11.853035,4.605301
5146,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{},None,None,0.0,8.075771,22.180904
5147,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'},japanese,None,0.0,8.734847,22.837754
5148,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne...",None,None,0.0,6.634839,20.700126


In [13]:
# number of chain restaurants with chosen distance of location 1
dist1_and_chain=osm_data[(osm_data.dist1<dist)&(osm_data.is_chain_restaurant==1)].shape[0]
dist1_and_chain

21

In [14]:
# number of chain restaurants with chosen distance of location 2
dist2_and_chain=osm_data[(osm_data.dist2<dist)&(osm_data.is_chain_restaurant==1)].shape[0]
dist2_and_chain

154

In [15]:
# number of non chain restaurants with chosen distance of location 1
dist1_and_nonchain=osm_data[(osm_data.dist1<dist)&(osm_data.is_chain_restaurant==0)].shape[0]
dist1_and_nonchain

116

In [16]:
# number of non chain restaurants with chosen distance of location 2
dist2_and_nonchain=osm_data[(osm_data.dist2<dist)&(osm_data.is_chain_restaurant==0)].shape[0]
dist2_and_nonchain

1671

In [17]:
location = {
            'location 1': [dist1_and_nonchain, dist2_and_nonchain],
            'location 2': [dist1_and_chain, dist2_and_chain]
        }
chi=pd.DataFrame(location, 
             columns = [ 'location 1', 'location 2'],
             index=['chain restaurant', 'non chain restaurant'])
chi

,location 1,location 2
chain restaurant,116,21
non chain restaurant,1671,154


## VISUALIZATION

In [18]:
# put a marker on location 1 and 2 on map
m3=folium.Map(location=location2, zoom_start=100)
folium.Marker(location1, popup='<b>Location 1</b>').add_to(m3)
folium.Marker(location2, popup='<b>Location 2</b>').add_to(m3)

In [19]:
# select only restauarnts with distance of your chosen distance
within_distance=osm_data[(osm_data.dist2<dist)|(osm_data.dist1<dist)]
chain_restaurant = within_distance[within_distance.is_chain_restaurant==1][["lat","lon"]].values
non_chain_restaurant = within_distance[within_distance.is_chain_restaurant==0][["lat","lon"]].values


In [20]:
# blue for restaurants within chosen distance of location 1
for i in range(len(chain_restaurant)):
    folium.CircleMarker(chain_restaurant[i], radius=8, color='blue', fill=True).add_to(m3)
    

# red for restaurants within chosen distance of location 2   
for i in range(len(non_chain_restaurant)):
    folium.CircleMarker(non_chain_restaurant[i], radius=2, color='red', s=25, fill=True).add_to(m3)
m3.save('map.html')
display(m3)

In [21]:
m=folium.Map(location=location2, zoom_start=100)
folium.Marker(location1, popup='<b>SFU Burnaby</b>').add_to(m)
folium.Marker(location2, popup='<b>SFU Vancouver</b>').add_to(m)


In [22]:
latlons = osm_data[["lat","lon"]].values

HeatMap(latlons).add_to(m)
m

In [23]:
m.save("map_heatmap.html")

In [26]:
import folium
from folium.plugins import HeatMap
from haversine import haversine, Unit, haversine_vector
import pandas as pd

def make_matrix(location, size):
    return [location for i in range(size)]

def nonRestaurestaurant(): 
    return [
    'ATLAS_clean_room' ,'EVSE','Observation Platform' ,'Pharmacy','animal_shelter','arts_centre','atm','atm;bank',
    'bank','bench','bicycle_parking','bicycle_rental','bicycle_repair_station','car_rental','car_rep','car_sharing',
    'car_wash','casino','charging_station','childcare','chiropractor','cinema','clinic','clock','college',
    'community_centre','compressed_air','conference_centre','construction','courthouse','cram_school','dentist',
    'events_venue','family_centre','ferry_terminal','fire_station','first_aid','fountain','fuel','gambling','gym',
    'healthcare','hospital','housing co-op','hunting_stand','kindergarten','language_school','leisure','letter_box',
    'library','loading_dock','lobby','lounge','luggage_locker','marketplace','meditation_centre','monastery',
    'money_transfer','motorcycle_parking','motorcycle_rental','music_school','nightclub','nursery',
    'office|financial','park','parking','parking_entrance','parking_space','payment_terminal','pharmacy',
    'photo_booth','place_of_worship','playground','police','post_box','post_depot','post_office','prep_school',
    'public_bookcase','public_building','ranger_station','recycling','research_institute','safety',
    'sanitary_dump_station','school','science','scrapyard','seaplane terminal','shelter','shop|clothes','shower',
    'smoking_area','social_centre','social_facility','spa','storage','storage_rental','stripclub','studio',
    'taxi','telephone','theatre','toilets','townhall','training','trash','trolley_bay','university','vacuum_cleaner',
    'vending_machine','driving_school','veterinary','waste_basket','waste_disposal','waste_transfer_station',
    'water_point','watering_place','workshop''bureau_de_change','bus_station','internet_cafe', 'doctors',
    'dojo','bureau_de_change', 'boat_rental', 'workshop', 'drinking_water'

    ]
    

def main(file1, file2, location1, location2, dist):
    osm_data = pd.read_json(file1)
    
    # chain restaurant data using qid
    chain_qids=pd.read_json(file2)
    
    # merge osm_data and chain_qid to identify chain restaurants in 
    osm_data=osm_data.merge(chain_qids, how='left', on='qid')
    osm_data['is_chain_restaurant']=osm_data.is_chain_restaurant.fillna(0)
    
    # filter restaurants from non restaurant in osm_data
    amenity=osm_data['amenity']
    restaurant=list(dict.fromkeys([i for i in amenity if i not in nonRestaurestaurant()]))
    #print(restaurant)
    osm_data=osm_data[osm_data['amenity'].isin(restaurant)]
    
    size=osm_data.lat.size
    
        
    #convert locations list to matrix for haversine_vector(), distance calculation
    location1_matrix=make_matrix(location1, size)
    location2_matrix=make_matrix(location2, size)

    #distance between location 1 and osm_data locations
    osm_data['dist1']=haversine_vector(
        osm_data[['lat', 'lon']].values.tolist(), [location1 for i in range(size)], 
        Unit.KILOMETERS
    )

    #distance between location 2 and osm_data locations
    osm_data['dist2']=haversine_vector(
        osm_data[['lat', 'lon']].values.tolist(), [location2 for i in range(size)], 
        Unit.KILOMETERS
    )
    
    
    # number of chain restaurants with chosen distance of location 1
    dist1_and_chain=osm_data[(osm_data.dist1<dist)&(osm_data.is_chain_restaurant==1)].shape[0]
    
    # number of chain restaurants with chosen distance of location 2
    dist2_and_chain=osm_data[(osm_data.dist2<dist)&(osm_data.is_chain_restaurant==1)].shape[0]
    
    # number of non chain restaurants with chosen distance of location 1
    dist1_and_nonchain=osm_data[(osm_data.dist1<dist)&(osm_data.is_chain_restaurant==0)].shape[0]
    
    # number of non chain restaurants with chosen distance of location 2
    dist2_and_nonchain=osm_data[(osm_data.dist2<dist)&(osm_data.is_chain_restaurant==0)].shape[0]
    
    
   
    
    #Chi-Squared with the filtered data to compare the density of chain restaurants by two locations
    location = {
        'location 1': [dist1_and_nonchain, dist2_and_nonchain],
        'location 2': [dist1_and_chain, dist2_and_chain]
    }
    chi_squared=pd.DataFrame(location, 
        columns = [ 'location 1', 'location 2'],
        index=['chain restaurant', 'non chain restaurant']
    )
    print(chi_squared)
    
    
    # for map visualization
    
    # put a marker on location 1 and 2 on map
    m3=folium.Map(location=location2, zoom_start=100)
    folium.Marker(location1, popup='<b>Location 1</b>').add_to(m3)
    folium.Marker(location2, popup='<b>Location 2</b>').add_to(m3)
    
    
    # select only restauarnts with distance of your chosen distance
    within_distance=osm_data[(osm_data.dist2<dist)|(osm_data.dist1<dist)]
    chain_restaurant = within_distance[within_distance.is_chain_restaurant==1][["lat","lon"]].values
    non_chain_restaurant = within_distance[within_distance.is_chain_restaurant==0][["lat","lon"]].values
    
    
    # blue for restaurants within chosen distance of location 1
    for i in range(len(chain_restaurant)):
        folium.CircleMarker(chain_restaurant[i], radius=8, color='blue', fill=True).add_to(m3)
    

    # red for restaurants within chosen distance of location 2   
    for i in range(len(non_chain_restaurant)):
        folium.CircleMarker(non_chain_restaurant[i], radius=2, color='red', s=25, fill=True).add_to(m3)
        
    m3.save('map.html')
    display(m3)
    
    
    # For heat map visualization - with similar procedure
    m=folium.Map(location=location2, zoom_start=100)
    folium.Marker(location1, popup='<b>SFU Burnaby</b>').add_to(m)
    folium.Marker(location2, popup='<b>SFU Vancouver</b>').add_to(m)
    latlons = osm_data[["lat","lon"]].values
    HeatMap(latlons).add_to(m)
    m.save('heat_map.html')
    display(m)
    



if __name__ == '__main__':
    # default intializations
    file1='data/preprocessed-osm-data.json.gz'
    file2='data/chain-restaurant-qids.json'
    
    while True:
    
        flag=int(input("Choose:\n0 - to enter your own values\n1 - to use default values: \n"))
        
    
    
    
        if flag==0:

            print("Enter coordinations for location 1:\n")
            lat1=float(input("Enter latitude 1: "))
            lon1=float(input("Enter Longitude 1: "))


            print("Enter coordinations for location 2:\n")
            lat2=float(input("Enter latitude 2: "))
            lon2=float(input("Enter Longitude 2: "))

            dist=float(input("Enter the distance within location 1 and 2 you are interested in(km): "))

            location1 = [lat1,  lon1]
            location2 = [lat2,  lon2]

            print('\n\n\n')
            main(file1, file2, location1, location2, dist)
            break

        if flag==1:

            location1 = [49.2768,  -122.9180] #SFU Burnaby
            print('location 1(SFU Burnaby): ', location1)

            location2 = [49.284478, -123.112349]  #SFU Vancouver
            print('location 2(SFU Vancouver): ', location2)

            dist=5
            print('distance within location 1 and 2  interested in: ', dist,'km')

            print('\n\n\n')
            main(file1, file2, location1, location2, dist)
            break
        
        
    

Choose:
0 - to enter your own values
1 - to use default values: 
0
Enter coordinations for location 1:

Enter latitude 1: 49
Enter Longitude 1: -100
Enter coordinations for location 2:

Enter latitude 2: 30
Enter Longitude 2: -125
Enter the distance within location 1 and 2 you are interested in(km): 10




                      location 1  location 2
chain restaurant               0           0
non chain restaurant           0           0
